### Wigner function positivity

In [1]:
from sage.all import *
import numpy as np

In [2]:
# Build the field, the Galois ring and the Teichmüller set

p = 2
N = 3 # number of qubits

F = GF(p**N, 'x')
x = F.gen()

R = PolynomialRing(Integers(4), 't')
t = R.gen()

GR = R.quotient(t**3 + 2*t**2 + t - 1, 'w')
w = GR.gen()

# Get field element index
def toInt(k):
    return list(F).index(k)

# Teichmüller set
T = [GR(0)] + [w**j for j in range(1, 2**N - 1)] + [GR(1)]

# Lift a field element to its Teichmüller representative
def TeichLift(k):
    return T[toInt(k)]
hat = TeichLift # shortcut

In [3]:
# Now build the group character and the generalized Pauli operators

def chi(k):
    return np.exp(np.pi * 1j * int(k.trace()))

def Proj(u, v=None):
    if v is None:
        v = u
    return np.outer(u, v.conj().T)

Id = np.eye(2**N, 2**N)

def Fourier():
    s = np.zeros((2**N, 2**N), dtype='complex128')
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / np.sqrt(2**N)
    return s
FF = Fourier()

def Z(a):
    return np.diag([chi(a * k) for k in F])

def X(b):
    return FF.conj().T @ Z(b) @ FF

# Sainz's phase uses the Galois ring trace down to Z_4. 
# SageMath handles the 2-adic decomposition to define the trace.
def phi(a, b):
    if not isinstance(a, type(w)):
        a = hat(a)
    if not isinstance(b, type(w)):
        b = hat(b)

    return (1j)**(int((a * b).trace()))

def D(a, b):
    return phi(a, b) * (Z(a) @ X(b))

In [4]:
# Definiion of the rotation operators for the Desarguesian spread.
# The function gives the full basis corresponding to a given parameter \mu.
def V(mu):
    s = np.zeros((2**N, 2**N), dtype='complex128')
    for i, k in enumerate(F):
        s += phi(k, mu * k) * Proj(FF[:, i])
    return s

# We can obtain the whole set of MUBs, ordered
# by the field elements in the following way:
mubs306 = [FF] + [V(mu) for mu in F]

In [6]:
# Wootters kernel for Desarguesian spread
def Wootters(a, b):
    op = Proj(mubs306[0][:, toInt(a)]) # Fourier basis
    for xi in F:
        # for nu in F:
        #     d = float(b == xi * a + nu)
        #     op += d * Proj(mubs306[toInt(xi)+1][:,toInt(nu)])
        nu = b - xi * a
        op += Proj(mubs306[toInt(xi)+1][:,toInt(nu)])
    return op - Id

# Definition of the Wigner function
def Wigner(state, a, b, kernel):
    return (state @ kernel(a, b)).trace()

def WignerMatrix(state, kernel):
    W = np.zeros((2**N, 2**N))
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            W[i, j] = np.real(Wigner(state, a, b, kernel)) / (2**N)
    return W

# shortcut
def WW(state, approx=True):
    w = WignerMatrix(Proj(state), Wootters)
    if approx:
        return np.round(w, 3)
    return w

# Sum the Wigner function along a given curve
def Prob(w, curve):
    s = 0
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s += w[i,j] * float(b == curve(a))
    return s

Now let's build the MUBs for the $(1,6,2)$ factorization curves. The curves are parametrized as 
$$f_\mu(\alpha) = \mu \alpha + \alpha^2 + \alpha^4,$$
for $\mu \in F$.

In [7]:
def f(mu):
    return lambda k: mu * k + k**2 + k**4

In [8]:
# distributivity
for mu in F:
    for a in F:
        for b in F:
            if f(mu + a)(b) != f(mu)(b) + f(a)(b):
                raise Exception('Curves do not distribute!')

Exception: Curves do not distribute!

Note that this curves do *not* in fact define a presemifield as left distributivity does not hold if the the operation $\circ$ is defined as $\mu \circ \alpha = f_\mu(\alpha)$. How does this effect the proof? It does not seem to be problematic since the main part of the proof just requires that
$$\chi(\alpha f_\mu(\xi)) = \chi(\xi f_\mu(\alpha)),$$
which can be shown to be true.

In [9]:
for mu in F:
    for a in F:
        for xi in F:
            if chi(a * f(mu)(xi)) != chi(xi * f(mu)(a)):
                raise Exception('Property does not hold!')

Now we will numerically prove (for a given example of curves) Sainz's proof of the positivity of the Wigner function for eigenstates built out of rotation operators. First we must obtain the bases.

In [10]:
def f_hat(mu):
    return lambda k: hat(mu * k) + hat(k**2) + hat(k**4)

In [11]:
def V2(mu):
    s = np.zeros((2**N, 2**N), dtype='complex128')
    for i, k in enumerate(F):
        s += phi(k, f_hat(mu)(k)) * Proj(FF[:, i])
    return s

In [12]:
mubs162 = [FF] + [V2(mu) for mu in F]

Now we try to verify the equations (40) to (42). Equation (40) gives us
$$
|\langle \psi_{\mu, \beta - \mu \alpha} | \psi_\kappa^{f_\nu} \rangle|^2
= \frac{1}{d} \sum_\xi c_{\xi, f_\nu(\xi)} c_{\xi, \mu\xi}^* \chi(\xi(-\kappa+\beta-\mu\alpha))
\delta_{\mu\xi, f_\nu(\xi)}.
$$

In [13]:
# Verification of the equation (40).

for alpha in F:
    for beta in F:
        for s in F:        
            for k in F:
                for mu in F:
                    nu = beta - mu * alpha
    
                    psi_mu_nu = mubs306[toInt(mu)+1][:,toInt(nu)]
                    psi_f_k = mubs162[toInt(s)+1][:,toInt(k)]
    
                    inp = abs(psi_mu_nu.conj() @ psi_f_k)**2
    
                    ss = 0
                    for xi in F:
                        c_xif = phi(xi, f_hat(s)(xi))
                        c_ximu = phi(xi, xi * mu)
                        char = chi(xi * (-k + nu))
                        d = int(mu * xi == f(s)(xi))
                        ss += c_xif * conjugate(c_ximu) * char * d / (2**N)
                    
                    if not np.isclose(inp, ss):
                        raise Exception('No equality!', mu, nu, s, k)
print('Equation (40) is valid for all (a,b) and all curves!')

Equation (40) is valid for all (a,b) and all curves!


The next, and crucial part of the proof is summing over the parameter $\mu$, which means we find all of the straight lines that pass through the point $(\alpha, \beta)$. Since the last equation is verified for all possible combinations of parameters, simply summing the left and right hand side of equation (40) doesn't prove much. What we need to do is sum the left hand side and show that it is equal to the right hand side of equation (42):
$$
\sum_\mu |\langle \psi_{\mu, \beta - \mu \alpha} | \psi_\kappa^{f_\nu} \rangle|^2
= 1 + \frac{1}{d} \sum_{\xi \in F^*} \chi(\xi(-\kappa + \beta - f_\nu(\alpha))),
$$
where $F^* = F \setminus \{0\}$. This as we shall now see, seems to fail.

In [119]:
# Equation (42)

alpha = x
beta  = x
for s in F:
    for k in F:
        inps = 0
        s40 = 0
        for mu in F:
            nu = beta - mu * alpha

            # Inner product sum
            psi_mu_nu = mubs306[toInt(mu)+1][:,toInt(nu)]
            psi_f_k = mubs162[toInt(s)+1][:,toInt(k)]

            inps += np.abs(psi_mu_nu.conj() @ psi_f_k)**2

            # Right hand side of (40)
            for xi in F:
                c_xif = phi(hat(xi), curve_hat(xi))
                c_ximu = phi(xi, mu * xi)
                char = chi(xi * (-k + beta - mu * alpha))
                d = int(mu * xi == f(s)(xi))
                s40 += c_xif * conjugate(c_ximu) * char * d / (2**N)

        # Right hand side of (42)
        s42 = 1
        for xi in list(F)[1:]:
            s42 += chi(xi * (k + beta + f(s)(alpha))) / (2**N)
        
        print(np.round(inps, 4), np.round(s40, 4), np.round(s42, 4), s, k)
        if not np.isclose(inps, s42):
            raise Exception('Sums do not match!')

0.875 (0.875+0j) (1.875+0j) 0 0


Exception: Sums do not match!

So the sums already fail for $\alpha = x$, $\beta = x$ and $s = 0$, $k = 0$. Let's investigate what is going on. Equation (40) becomes
$$
\sum_\mu |\langle \psi_{\mu, x - \mu x}|\psi_0^{f_0}\rangle|^2
= 
\frac{1}{d} \sum_\mu \sum_\xi c_{\xi, f_0(\xi)} c_{\xi, \mu\xi}^* \chi(\xi(x-\mu x))
\delta_{\mu\xi, f_0(\xi)},
$$
where $f_0(k) = k^2 + k^4$. Notice that 
$$
\mu \xi = f_0(\xi) = \xi^2 + \xi^4
$$
is satisfied trivially for $\xi = 0$, and for $\xi \neq 0$ it is satisfied if and only if $\mu = \xi + \xi^3$. 

In [122]:
s = 0
for mu in F:
    for xi in F:
        c_xif = phi(hat(xi), f_hat(F(0))(xi))
        c_ximu = phi(xi, mu * xi)
        char = chi(xi * (x - mu * x))
        d = float(mu * xi == f(F(0))(xi))
        s += c_xif * conjugate(c_ximu) * char * d / (2**N)
s

(0.875+0j)

So the actual sum is right, that means the formula in equation (42) has to be wrong in some way. This is probably down to the fact that $f_\mu$ does not produce a presemifield operation.

In [127]:
for mu in F:
    for xi in F:
        if mu * xi == f(F(0))(xi):
            print(mu, ',', xi)

0 , 0
0 , 1
x , 0
x^2 , 0
x + 1 , 0
x + 1 , x^2 + 1
x^2 + x , 0
x^2 + x + 1 , 0
x^2 + x + 1 , x + 1
x^2 + 1 , 0
x^2 + 1 , x^2 + x + 1
1 , 0
1 , x
1 , x^2
1 , x^2 + x


As we can see wehave 15 possible solutions to the equation $\mu \xi = f_0(\xi)$. 

For $\xi = 0$ we have 8 solutions. For $\xi \neq 0$, we have $\mu = \xi + \xi^3 = \xi^{-1} f_0(\xi)$. If $f_0$ were one to one, then we would have a different value for $\mu$, but as we can verify, this equation only has 5 unique solutions:

In [137]:
[(xi, xi + xi**3) for xi in list(F)[1:]]

[(x, 1),
 (x^2, 1),
 (x + 1, x^2 + x + 1),
 (x^2 + x, 1),
 (x^2 + x + 1, x^2 + 1),
 (x^2 + 1, x + 1),
 (1, 0)]

And so we obtain:
$$
\begin{align*}
\frac{1}{d} \sum_\mu \sum_\xi c_{\xi, f_0(\xi)} c_{\xi, \mu\xi}^* \chi(\xi(x-\mu x))
\delta_{\mu\xi, f_0(\xi)}
&= \frac{1}{d} \left(
\sum_\mu c_{0, f_0(0)} c_{0, 0}^* \chi(0) \delta_{0, f_0(0)}
+
\sum_\mu \sum_{\xi \in F^*} c_{\xi, f_0(\xi)} c_{\xi, \mu\xi}^* \chi(\xi(x-\mu x))
\delta_{\mu\xi, f_0(\xi)}
\right) \\
&= \frac{1}{d} \left(
d
+
\sum_{\xi \in F^*} c_{\xi, f_0(\xi)} c_{\xi, (\xi+\xi^3)\xi}^* \chi(\xi(x-(\xi+\xi^3) x))
\right) 
\end{align*}
$$

In [161]:
t1 = 0
t2 = 0
for mu in F:
    c_xif = phi(F(0), f_hat(F(0))(F(0)))
    c_ximu = phi(F(0), F(0))
    char = chi(F(0))
    d = int(mu * F(0) == f(F(0))(F(0)))
    t1 += c_xif * conjugate(c_ximu) * char * d / (2**N)
    
    for xi in list(F)[1:]:
        c_xif = phi(xi, f_hat(F(0))(xi))
        c_ximu = phi(xi, mu * xi)
        char = chi(xi * (x - mu * x))
        d = int(mu * xi == f(F(0))(xi))
        if d:
            print(mu, '|', xi+xi**3, '|', f(F(0))(xi), c_xif, c_ximu, char)
        t2 += c_xif * conjugate(c_ximu) * char * d / (2**N)
        
(t1, t2)

0 | 0 | 0 (-1+0j) (1+0j) (1+0j)
x + 1 | x + 1 | x^2 (1+0j) (-1+0j) (1+0j)
x^2 + x + 1 | x^2 + x + 1 | x (1+0j) (-1+0j) (1+0j)
x^2 + 1 | x^2 + 1 | x^2 + x (1+0j) (-1+0j) (1+0j)
1 | 1 | x (-1+0j) (-1+0j) (1+0j)
1 | 1 | x^2 (-1+0j) (-1+0j) (1+0j)
1 | 1 | x^2 + x (-1+0j) (-1+0j) (1+0j)


((1+0j), (-0.125+0j))

I think the problem reduces to the value $c_{\xi, \mu \xi}^*$ when $\mu = \xi + \xi^3$. By definition we have
$$
\begin{align*}
\Phi(\xi, \mu \xi) &= \omega[\hat{\xi} \widehat{\mu \xi}] \\
&= \omega[\hat\xi \widehat{\xi+\xi^3} \hat \xi] \\
&= \omega[\hat\xi (\hat{\xi}+\hat{\xi}^3 + \sqrt{\hat\xi \hat\xi^3}) \hat \xi] \\
&= \omega[\hat\xi (\hat{\xi}^2+\hat{\xi}^4 + \hat\xi^3)]
\end{align*}
$$
But by lifting the operation we have:
$$
\begin{align*}
\Phi(\xi, f_0(\xi)) &= \omega[\hat\xi f_{\hat 0}(\hat\xi)] \\
&= \omega[\hat\xi (\hat\xi^2 + \hat\xi^4)]
\end{align*}
$$
And so the rotation coefficients are distinct and therefore do not cancel out by conjugation.

In [169]:
for xi in F:
    mu = xi + xi**3
    print(phi(xi, mu * xi), phi(xi, f_hat(0)(xi)))

(1+0j) (1+0j)
(-1+0j) (-1+0j)
(-1+0j) (-1+0j)
(-1+0j) (1+0j)
(-1+0j) (-1+0j)
(-1+0j) (1+0j)
(-1+0j) (1+0j)
(1+0j) (-1+0j)


So the problem seems to stem from the fact that although $\mu \xi = f_\nu(\xi)$ for some $\mu, \nu$ and $\xi$, the phase may still be distinct since we use $f_{\hat\nu}(\hat\xi)$ and not $\widehat{f_\nu(\xi)}$ to define the curve phase.

In [172]:
# Kernel and Wigner function testing

from utils import checkPhasePointOperators

class TestSuite:

    def __init__(self):
        pass

    def run(self):
        self.RotationOperators()
        self.RotationCoefficients()
        self.KernelProperties()
        self.TomographicProperties()
        self.Covariance()

        print('All tests have passed successfully!')

    def RotationOperators(self):
        try:
            for mu in F:
                for alpha in F:
                    v_op = V(mu)
                    z_op = Z(alpha)
                    rot_op = v_op @ z_op @ v_op.conj().T
                    disp = D(alpha, mu * alpha)
                    if not np.all(np.isclose(rot_op, disp)):
                        raise Exception('Rotation failed!') 
        finally:
            print('Rotation operators work!')

    def RotationCoefficients(self):
        try:
            for mu in F:
                for k in F:
                    for a in F:
                        c_k = phi(k, k * mu)
                        c_a = phi(a, a * mu)
                        c_ak = phi(k + a, (k + a) * mu)
                        char = chi(mu * a * k)
                        if not np.isclose(c_k * c_a, c_ak * char):
                            raise Exception(
                                'Recurrence relation does not hold!',
                            mu, k, a)
        finally:
            print('Recurrence relation holds for chosen phase!')

    def KernelProperties(self):
        ops = []
        for a in F:
            for b in F:
                ops.append(Wootters(a, b))
        
        checkPhasePointOperators(ops)
        
        if np.all(np.isclose(sum(ops) / 2**N, Id)):
            print('Kernel is normalized!')

    def TomographicProperties(self):
        try:
            # Vertical and horizontal lines
            for nu in F:
                op = np.zeros((2**N, 2**N), dtype='complex128')
                for a in F:
                    op += Wootters(a, nu)
                if not np.all(np.isclose(op / 2**N, Proj(Id[:, toInt(nu)]))):
                    raise Exception('Error with Z eigenbasis!')
                
                op = np.zeros((2**N, 2**N), dtype='complex128')
                for b in F:
                    op += Wootters(nu, b)
                if not np.all(np.isclose(op / 2**N, Proj(FF[:, toInt(nu)]))):
                    raise Exception('Error with X eigenbasis!')
                
            # Arbitrary curve
            for mu in F:
                for nu in F:
                    op = np.zeros((2**N, 2**N), dtype='complex128')
                    for a in F:
                        for b in F:
                            op += Wootters(a, b) * float(b == mu * a + nu)
                    v = mubs306[toInt(mu)+1][:,toInt(nu)]
                    if not np.all(np.isclose(op / 2**N, Proj(v))):
                        raise Exception('Error with Z_a X_f(a) eigenbasis!')
        finally:
            print('Tomographic properties hold!')
    
    def Covariance(self):
        try:
            for k in F:
                for l in F:
                    for a in F:
                        for b in F:
                            disp = D(k, l)
                            disp_op = disp @ Wootters(a, b) @ disp.conj().T
                            if not np.all(np.isclose(disp_op, Wootters(a+k, b+l))):
                                raise Exception('Not covariant', a, b, k, l)
        finally:
            print('Kernel is covariant!')

    # Testing tomographic probabilities

    def TestProb(self, state):
        w = WW(state, approx=False)
        for mu in F:
            for nu in F:
                wigner_prob = Prob(w, lambda t: mu * t + nu)
                trans_prob  = abs(
                    state.conj() @ mubs306[toInt(mu)+1][:,toInt(nu)]
                )**2
                if not np.isclose(wigner_prob, trans_prob):
                    raise Exception(
                        'Probabilities do not match!',
                        mu, nu, wigner_prob, trans_prob
                    )
        print('Probabilities are correct!')

In [173]:
test = TestSuite()
test.run()

Rotation operators work!
Recurrence relation holds for chosen phase!
Operators are self-adjoint and unit trace!
Operators are orthonormal!
Kernel is normalized!
Tomographic properties hold!
Kernel is covariant!
All tests have passed successfully!
